# CSE-CIC-IDS 2017 Logistic Regression

In [1]:
model_id = "logisticregression"

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import glob

In [3]:
NOTEBOOK_PATH = "C:/Users/Xetrov/Desktop/SciFair20/Code/"

In [4]:
x_scaled = pd.read_csv(NOTEBOOK_PATH + "IDS2017/x_scaled_powertransform.csv")

y_df_enc = pd.read_csv(NOTEBOOK_PATH + "IDS2017/y_grouped_1henc.csv")

# Split data into train and test

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
x_train, x_valtest, y_train, y_valtest = train_test_split(x_scaled, y_df_enc, test_size = 0.4, random_state = 42)

In [7]:
x_val, x_test, y_val, y_test = train_test_split(x_valtest, y_valtest, test_size = 0.5, random_state = 42)

In [8]:
del x_valtest 
del y_valtest

In [9]:
print("Train:")
print(y_train.sum(axis=0))
print("\nValidation:")
print(y_val.sum(axis=0))
print("\nTest:")
print(y_test.sum(axis=0))

Train:
BENIGN          1364100
Botnet             1185
Brute Force        8273
DoS/DDoS         228292
Infiltration         24
PortScan          95267
Web Attack         1304
dtype: int64

Validation:
BENIGN          454207
Botnet             397
Brute Force       2810
DoS/DDoS         76510
Infiltration         7
PortScan         31787
Web Attack         431
dtype: int64

Test:
BENIGN          454790
Botnet             384
Brute Force       2752
DoS/DDoS         75897
Infiltration         5
PortScan         31876
Web Attack         445
dtype: int64


# Train model

In [10]:
from sklearn.linear_model import LogisticRegression

In [11]:
%%time
clf = LogisticRegression(solver="saga", multi_class="multinomial", verbose=2, class_weight=None, n_jobs=-1).fit(x_train, y_train.to_numpy().argmax(axis=1))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.


max_iter reached after 412 seconds


C:\Users\Xetrov\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  6.9min finished


Wall time: 6min 53s


**Save model**

In [12]:
import pickle

In [13]:
import time

model_filename = model_id  + " [" + time.strftime("%Y%m%d %H%M") + "]"

In [14]:
save_file = open(NOTEBOOK_PATH + "Models/" + model_filename + ".pkl", "wb")
saved_model = pickle.dump(clf, save_file)
save_file.close()

**Test model**

In [22]:
pred = clf.predict(x_val)

In [25]:
pred_index = pred
# pred_index = list(pred.argmax(1))
y_index = list(np.argmax(y_val.to_numpy(), axis=1))

atktypes = sorted(y_val.columns)
pred_series = pd.Series(pred_index, name="Pred").replace(dict(enumerate(atktypes)))
y_series = pd.Series(y_index, name="Actual").replace(dict(enumerate(atktypes)))

matrix = pd.crosstab(pred_series, y_series, margins=True)
matrix

Actual,BENIGN,Botnet,Brute Force,DoS/DDoS,Infiltration,PortScan,Web Attack,All
Pred,,,,,,,,
BENIGN,384076,1,0,169,0,81,0,384327
Botnet,22596,396,0,0,0,0,0,22992
Brute Force,3880,0,2806,205,0,63,0,6954
DoS/DDoS,8001,0,4,75415,0,46,4,83470
Infiltration,18486,0,0,99,7,0,0,18592
PortScan,3734,0,0,5,0,31536,0,35275
Web Attack,13434,0,0,617,0,61,427,14539
All,454207,397,2810,76510,7,31787,431,566149


# Test Set

In [15]:
from sklearn.metrics import f1_score

In [16]:
pred = clf.predict(x_test)
pred_index = pred
# pred_index = list(pred.argmax(1))
y_index = list(np.argmax(y_test.to_numpy(), axis=1))
f1_micro = f1_score(y_index, pred_index, average='micro')
f1_macro = f1_score(y_index, pred_index, average='macro')

print(f1_micro)
print(f1_macro)

C:\Users\Xetrov\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.9845870963297648
0.5038259382707292


In [17]:
atktypes = sorted(y_val.columns)
pred_series = pd.Series(pred_index, name="Pred").replace(dict(enumerate(atktypes)))
y_series = pd.Series(y_index, name="Actual").replace(dict(enumerate(atktypes)))

matrix = pd.crosstab(pred_series, y_series, margins=True)
matrix.to_csv(NOTEBOOK_PATH + "Confusion Matrices/test_" + model_filename + ".csv")
matrix

Actual,BENIGN,Botnet,Brute Force,DoS/DDoS,Infiltration,PortScan,Web Attack,All
Pred,,,,,,,,
BENIGN,450650,384,1162,1885,5,242,440,454768
Brute Force,272,0,1296,102,0,7,0,1677
DoS/DDoS,3422,0,294,73905,0,55,5,77681
PortScan,446,0,0,5,0,31572,0,32023
All,454790,384,2752,75897,5,31876,445,566149


# Log results

In [34]:
model_log = open(NOTEBOOK_PATH + "model_log.txt", "a")

model_log.write("\n" + model_filename)
model_log.write("\n\tF1 Micro: " + str(f1_micro))
model_log.write("\n\tF1 Macro: " + str(f1_macro))

model_log.close()